In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import MDS

path = "/home/ubuntu/onekgenomes/"
sampleDataFile = "data/sampleData/sampleData.tsv"
df = pd.read_csv(path + sampleDataFile, sep='\t', index_col=0)

In [ ]:
pops = df["Population"].tolist()
uniquePops = list(set(pops))
uniquePops.sort()
numSamples = len(pops)
targetDict = dict(zip(uniquePops, range(numSamples)))
Y = np.array([targetDict[pop] for pop in pops])

In [ ]:
superPops = df["Super Population"].tolist()
uniquePops = list(set(superPops))
uniquePops.sort()
numSamples = len(superPops)
targetDict = dict(zip(uniquePops, range(numSamples)))
Y = np.array([targetDict[p] for p in superPops])

In [ ]:
genders = df["Gender"]
G = (genders == "male").values

In [ ]:
pdistMat = np.load(path + "data/pdist/summedMats/pdistAll.npy")
D_all = np.sqrt(pdistMat)
print("A")
numComp = 4
embedding = MDS(dissimilarity="precomputed", n_components=numComp)
embeddedData = embedding.fit_transform(D_all)
print(embeddedData.shape)
x, y, z, w = np.hsplit(embeddedData, numComp)

In [ ]:
pdistMat = np.load(path + "data/pdist/summedMats/pdistNum.npy")
D_num = np.sqrt(pdistMat)
numComp = 3
embedding = MDS(dissimilarity="precomputed", n_components=numComp)
embeddedData = embedding.fit_transform(D_num)
embeddedData.shape
x1, y1, z1 = np.hsplit(embeddedData, numComp)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax1 = fig.add_subplot(221, projection="3d")
cmap = cm.rainbow(Y / np.max(Y))
ax1.scatter(x, y, z, c=cmap)
ax2 = fig.add_subplot(222, projection="3d")
genderCmap = np.zeros((numSamples,))
genderCmap[G] = 1
ax2.scatter(x, y, z, c=genderCmap)

"""ax3 = fig.add_subplot(223, projection="3d")
ax3.scatter(x1, y1, z1, c=cmap)

fig = plt.figure()
ax = fig.add_subplot(111)
cmap = cm.rainbow(Y / np.max(Y))
ax.scatter(x, y, c=cmap)"""

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig2 = plt.figure()
ax4 = fig2.add_subplot(121)
cmap = cm.rainbow(Y / np.max(Y))
ax4.scatter(y,z, c=cmap)
ax4.set_xlim(-0.0015,0.0015)
ax4.set_ylim(-0.0015,0.0015)
ax5 = fig2.add_subplot(122)
ax5.scatter(x1, y1, c=cmap)
ax5.set_xlim(-0.0015,0.0015)
ax5.set_ylim(-0.0015,0.0015)

In [ ]:
from numpy.linalg import svd

D_sq = pdistMat
n = numSamples
K_c = -(np.eye(n) - np.ones((n, n)) / n) @ D_sq @ (np.eye(n) - np.ones((n, n)) / n)
U, S, _ = svd(K_c)

In [ ]:
S = np.diag(S)
A = U @ S
numComp = 3
x, y, z = np.hsplit(A[:,:numComp], numComp)

Drafting the script:

In [ ]:
import pandas as pd
import numpy as np

def processList(lst):
    numSamples = len(pops)
    uniqueElems = list(set(lst))
    uniqueElems.sort()
    targetDict = dict(zip(uniqueElems, range(numSamples)))
    return np.array([targetDict[elem] for elem in lst])

path = "/home/ubuntu/onekgenomes/"
sampleDataFile = "data/sampleData/sampleData.tsv"
df = pd.read_csv(path + sampleDataFile, sep='\t', index_col=0)
pops, superPops = processList(df["Population"].tolist()), processList(df["Super Population"].tolist())


In [ ]:
import numpy as np
from numpy.linalg import svd

def dual_pca(D_sq, k):
    m = D_sq.shape[0]
    gram = -(np.eye(m) - 1 / m) @ D_sq @ (np.eye(m) - 1 / m)
    U, Sigma_sq, _ = svd(gram)
    Sigma_k = np.sqrt(Sigma_sq[:k])
    return U[:,:k] * Sigma_k

In [1]:
import numpy as np
from numpy.linalg import svd

def complete_pca(D_sq):
    m = D_sq.shape[0]
    gram = -(np.eye(m) - 1 / m) @ (D_sq / 2) @ (np.eye(m) - 1 / m)
    U, Sigma_sq, _ = svd(gram)
    Sigma = np.diag(np.sqrt(Sigma_sq))
    return U @ Sigma

In [2]:
path = "/home/ubuntu/onekgenomes/"
D_sq = np.load(path + "data/pdist/summedMats/pdistAll.npy")
embedded = complete_pca(D_sq)
saveFile = path + "data/dimReduc/completePCA/embeddedAll.npy"
np.save(saveFile, embedded)

In [3]:
path = "/home/ubuntu/onekgenomes/"
D_sq = np.load(path + "data/pdist/summedMats/pdistNum.npy")
embedded = complete_pca(D_sq)
saveFile = path + "data/dimReduc/completePCA/embeddedNum.npy"
np.save(saveFile, embedded)

In [ ]:
path = "/home/ubuntu/onekgenomes/"
D_sq = np.load(path + "data/pdist/summedMats/pdistAll.npy")
numComp = 3
embedded = dual_pca(D_sq, numComp)
saveFile = path + "data/dimReduc/dualPCA/embeddedAll.npy"
np.save(saveFile, embedded)

In [ ]:
D_sq = np.load(path + "data/pdist/summedMats/pdistNum.npy")
numComp = 3
embedded = dual_pca(D_sq, numComp)
saveFile = path + "data/dimReduc/dualPCA/embeddedNum.npy"
np.save(saveFile, embedded)